In [1]:
import pandas as pd
df = pd.read_csv('../data/data_merged_with_scores.csv', sep=';')

In [3]:
df['source'].value_counts().sort_index() 

source
GAS                                                               303
GPT / https://www.anadventurousworld.com/usa-trivia-questions/     14
GPT / https://www.beelovedcity.com/england-quiz                    33
GPT-4 generated                                                   567
WVS                                                                84
Name: count, dtype: int64

In [4]:
df['category_group'].value_counts().sort_index() 

category_group
Economy, Health, and Environment              178
Lifestyle, Entertainment, and Daily Living    285
Media, Technology, and Education              141
Politics and Governance                       206
Society, Culture, and Relationships           191
Name: count, dtype: int64

In [5]:
df['question type'].value_counts().sort_index() 

question type
Binary Choice      475
Likert Scale       292
Multiple Choice    131
Numerical Scale     18
Ordinal Scale       38
Quiz question       47
Name: count, dtype: int64

In [6]:
mean_score_ground_truths = df['similarity_ground_truth_answers_uk_us'].mean()
mean_score_us = df['overall_score_us'].mean()
mean_score_uk = df['overall_score_uk'].mean()
print("Similarity GPT-US: " + str(mean_score_us), 
      "\nSimilarity GPT-UK: " + str(mean_score_uk), 
      "\nSimilarity UK-US: " + str(mean_score_ground_truths))
print("Number of q,a,a triples: " + str(len(df)))

Similarity GPT-US: 0.7404109597490045 
Similarity GPT-UK: 0.7113406496108827 
Similarity UK-US: 28.43867951781048
Number of q,a,a triples: 1002


In [7]:
print("Avg. number of words per question: " + str(average_word_count(df, 'question')))
print("Avg. number of words per answer: " + str((average_word_count(df, 'answer_us') + average_word_count(df, 'answer_uk')) / 2))
print("Unique words dataset: " + str(unique_word_count(df, 'question')))

print("GOQA - Avg. number of words per question: " + str(average_word_count(goqa, 'question')))
print("GOQA - Avg. number of words per answer: " + str((average_word_count(goqa, 'answer_us') + average_word_count(goqa, 'answer_uk')) / 2))
print("GOQA - Unique words dataset: " + str(unique_word_count(goqa, 'question')))


NameError: name 'average_word_count' is not defined

In [ ]:
def average_word_count(df, column_name):
    # Convert the column to strings
    df[column_name] = df[column_name].astype(str)
    # Calculate the number of words for each row in the column
    word_counts = df[column_name].apply(lambda x: len(x.split()))
    # Return the average word count
    return word_counts.mean()

In [8]:
def unique_word_count(df, column_name):

    # Convert the column to strings
    df[column_name] = df[column_name].astype(str)
    # Collect all words from each row in the column
    all_words = df[column_name].apply(lambda x: x.split()).explode()

    # Count unique words
    unique_words = set(all_words)

    return len(unique_words)

In [9]:
def correct_values(value):
    # Check if value is an integer or greater than 1
    if value > 1 or isinstance(value, int):
        return value / 1000
    else:
        return value

# Apply the correction function to the column
df['similarity_ground_truth_answers_uk_us'] = df['similarity_ground_truth_answers_uk_us'].apply(correct_values)

In [10]:
gpt_generated = df[df['source'] == 'GPT-4 generated']
mean_score_ground_truths = gpt_generated['similarity_ground_truth_answers_uk_us'].mean()
mean_score_us = gpt_generated['overall_score_us'].mean()
mean_score_uk = quiz['overall_score_uk'].mean()
print("Similarity GPT-US: " + str(mean_score_us), 
      "\nSimilarity GPT-UK: " + str(mean_score_uk), 
      "\nSimilarity UK-US: " + str(mean_score_ground_truths))
print("Number of q,a,a triples: " + str(len(quiz)))

Similarity GPT-US: 0.7832165178588248 
Similarity GPT-UK: 0.7346302110576967 
Similarity UK-US: 0.6852913584027972
Number of q,a,a triples: 567


In [11]:
goqa = df[(df['source'] == 'GAS') | (df['source'] == 'WVS')]
mean_score_ground_truths = goqa['similarity_ground_truth_answers_uk_us'].mean()
mean_score_us = goqa['overall_score_us'].mean()
mean_score_uk = goqa['overall_score_uk'].mean()
print("Similarity GPT-US: " + str(mean_score_us), 
      "\nSimilarity GPT-UK: " + str(mean_score_uk), 
      "\nSimilarity UK-US: " + str(mean_score_ground_truths))
print("Number of q,a,a triples: " + str(len(goqa)))

Similarity GPT-US: 0.6705235090639714 
Similarity GPT-UK: 0.6599592853051901 
Similarity UK-US: 0.6308118451079614
Number of q,a,a triples: 387


In [12]:
quiz = df[(df['source'] == 'GPT / https://www.beelovedcity.com/england-quiz') | (df['source'] == 'GPT / https://www.anadventurousworld.com/usa-trivia-questions/')]
mean_score_ground_truths = quiz['similarity_ground_truth_answers_uk_us'].mean()
mean_score_us = quiz['overall_score_us'].mean()
mean_score_uk = quiz['overall_score_uk'].mean()
print("Similarity GPT-US: " + str(mean_score_us), 
      "\nSimilarity GPT-UK: " + str(mean_score_uk), 
      "\nSimilarity UK-US: " + str(mean_score_ground_truths))
print("Number of q,a,a triples: " + str(len(quiz)))


Similarity GPT-US: 0.8017642517039116 
Similarity GPT-UK: 0.8564725924045482 
Similarity UK-US: 0.7081266597230383
Number of q,a,a triples: 47


In [240]:
import openai
import os
import pandas as pd
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def gpt4_text_similarity(text1, text2, model="gpt-4"):
    """
    Measures the similarity between two texts using GPT-4.

    Parameters:
    text1 (str): First text for comparison.
    text2 (str): Second text for comparison.
    model (str): The GPT model to use.

    Returns:
    float: A similarity score between 0 (not similar) and 1 (very similar).
    """

    system = f"On a scale between 1 and 5, how similar are the following two sentences? Respond only with a score between 1 and 5."
    prompt = f"Input:\n\n Sentence 1: {text1}\n\n Sentence 2: {text2}. \n Output:"
    #prompt = f"Rate the similarity between the following two texts on a scale from 0 (completely different) to 1 (identical):\n\nText 1: {text1}\n\nText 2: {text2}"
    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": system
        },
        {
        "role": "user",
        "content":  prompt
        }
    ],
    max_tokens=20
    )
    #print(response.choices[0].message.content)
    # Extracting the similarity score from the response
    try:
        last_message = response.choices[0].message.content
        similarity_score = float(last_message.strip())
    except (ValueError, KeyError, IndexError):
        similarity_score = None
    print(similarity_score)
    return similarity_score


In [246]:
#gpt_generated = data[data['source'] == 'GPT-4 generated']
quiz = pd.read_csv('quiz_answers_new.csv', sep=';')
def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['model_answer_uk'], row['answer_uk'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

quiz['gpt4_uk_score'] = quiz.apply(apply_similarity, axis=1)

5.0
5.0
5.0
5.0
4.5
1.0
5.0
5.0
5.0
2.0
5.0
5.0
4.0
5.0
5.0
5.0
5.0
5.0
4.0
5.0
5.0
5.0
5.0
1.0
5.0
4.0
5.0
5.0
5.0
5.0
4.5
5.0
5.0
1.0
5.0
5.0
5.0
5.0
1.0
2.0
5.0
5.0
5.0
5.0
5.0
1.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
1.0
5.0
4.5
5.0
5.0
5.0
5.0
4.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
1.0
1.0
1.0
1.0
4.0
2.0
5.0
1.0
1.0
5.0
4.5
4.5
1.0
1.0
5.0
5.0
5.0
5.0
5.0
1.0
4.0
5.0
1.0
5.0
1.0
1.0
5.0
1.0
1.0


In [247]:
print("US-Score Start")
def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['model_answer_us'], row['answer_us'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

quiz['gpt4_us_score'] = quiz.apply(apply_similarity, axis=1)

print("US-Score finished")

print("gpt4_usGT_ukGT_score start")
def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['answer_uk'], row['answer_us'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

quiz['gpt4_usGT_ukGT_score'] = quiz.apply(apply_similarity, axis=1)

print("gpt4_usGT_ukGT_score finished")

print("gpt4_ukGT_usMA_score start")
def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['answer_uk'], row['model_answer_us'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

quiz['gpt4_ukGT_usMA_score'] = quiz.apply(apply_similarity, axis=1)
print("gpt4_ukGT_usMA_score finished")

print("gpt4_usGT_ukMA_score start")

def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['answer_us'], row['model_answer_uk'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

quiz['gpt4_usGT_ukMA_score'] = quiz.apply(apply_similarity, axis=1)
print("gpt4_usGT_ukMA_score finished")

print("gpt4_usMA_ukMA_score start")
def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['model_answer_us'], row['model_answer_uk'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

quiz['gpt4_usMA_ukMA_score'] = quiz.apply(apply_similarity, axis=1)
print("gpt4_usMA_ukMA_score finished")


US-Score Start
5.0
2.0
5.0
4.5
2.0
1.0
1.0
5.0
5.0
4.5
5.0
5.0
2.0
5.0
5.0
5.0
5.0
5.0
1.0
1.0
1.0
5.0
1.0
1.0
5.0
5.0
5.0
4.5
1.0
1.0
1.0
5.0
5.0
1.0
5.0
5.0
5.0
5.0
1.0
2.0
3.0
5.0
5.0
5.0
1.0
1.0
5.0
2.0
5.0
1.0
5.0
5.0
1.0
5.0
1.0
1.0
5.0
5.0
1.0
5.0
4.5
5.0
5.0
5.0
5.0
1.0
5.0
5.0
4.0
2.0
1.0
1.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
2.0
1.0
5.0
2.0
5.0
5.0
5.0
5.0
5.0
1.0
1.0
5.0
5.0
5.0
5.0
2.0
1.0
5.0
5.0
1.0
5.0
1.0
1.0
5.0
US-Score finished
gpt4_usGT_ukGT_score start
1.0
2.0
5.0
2.0
5.0
1.0
1.0
1.0
1.0
1.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
1.0
1.0
5.0
5.0
5.0
5.0
1.0
1.0
4.5
5.0
5.0
1.0
5.0
5.0
5.0
5.0
1.0
1.0
5.0
5.0
2.0
4.0
1.0
1.0
5.0
1.0
5.0
1.0
4.0
5.0
1.0
5.0
5.0
1.0
5.0
4.5
1.0
5.0
4.0
4.5
4.5
5.0
5.0
5.0
1.0
5.0
1.0
5.0
1.0
5.0
5.0
5.0
1.0
5.0
1.0
1.0
5.0
1.0
5.0
5.0
1.0
5.0
5.0
5.0
5.0
3.0
5.0
4.5
2.0
5.0
1.0
1.0
1.0
1.0
5.0
5.0
1.0
5.0
1.0
1.0
1.0
gpt4_usGT_ukGT_score finished
gpt4_ukGT_usMA_score start
1.0
2.0
5.0
4.0
1.0
1.0
5.0
1.0
1.0
1.0
5.0
5.0
2.0
5.0
5.0

In [46]:
# Ground Truths
print(quiz['similarity_ground_truth_answers_uk_us'].mean())
print(quiz['gpt-4_score_ground_truths'].mean())

0.7081266597230383
0.6085106382978724


In [56]:
# US
print(gpt_generated['overall_score_us'].mean())
print(gpt_generated['gpt-4_us_score'].mean())

0.7832165178588248
0.6611640211640212


In [58]:
# Ground Truths
print(gpt_generated['similarity_ground_truth_answers_uk_us'].mean())
print(gpt_generated['gpt-4_score_ground_truths'].mean())

0.6852913584027972
0.5305996472663139


In [60]:
# UK Score
print(gpt_generated['overall_score_uk'].mean())
print(gpt_generated['gpt-4_uk_score'].mean())

0.7346302110576967
0.6012345679012345


In [61]:
def if_significantly_different(result1: list, result2: list, P_VALUE_THRES=0.05):
    from scipy import stats
    import numpy as np
    score, p_value = stats.ttest_ind(result1, np.array(result2), equal_var=False)
    if_sign = p_value <= P_VALUE_THRES
    return if_sign

In [62]:
Us_similarity_scores_per_sample = gpt_generated['gpt-4_us_score'].tolist()
Uk_similarity_scores_per_sample = gpt_generated['gpt-4_uk_score'].tolist()
if_significantly_different(Us_similarity_scores_per_sample,Uk_similarity_scores_per_sample )

True

In [63]:
eval_gpt_scores = gpt_generated[['answer_uk', 'answer_us', 'model_answer_uk', 'model_answer_us', 'gpt-4_uk_score', 'gpt-4_us_score', 'gpt-4_score_ground_truths']]
eval_gpt_scores.to_csv('eval_gpt_scores.csv')

Re-categorize question types in GOQA

In [71]:
import openai
import os
import pandas as pd
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def question_type(question, options_formatted, model="gpt-3.5-turbo"):
    """
    Measures the similarity between two texts using GPT-4.

    Parameters:
    text1 (str): First text for comparison.
    text2 (str): Second text for comparison.
    model (str): The GPT model to use.

    Returns:
    float: A similarity score between 0 (not similar) and 1 (very similar).
    """

    system = """Categorize the following question into one of the following categories. 
                First, decide if it is a scale, free-text or multiple choice question. 
                If it is a scale type question, determine if it is ordinal scale, likert scale or numerical scale. 
                For multiple choice, determine wether it is a binary choice or multiple choice question.
                Some questions have answer choices, some not.
                Examples: 
                Input: 
                question: Which statement comes closer to your own views, even if neither is exactly right? Using overwhelming military force is the best way to defeat terrorism around the world, Relying too much on military force to defeat terrorism creates hatred that leads to more terrorism
                options: 
                (A) Using overwhelming military force is the best way to defeat terrorism around the world
                (B) If (survey country) had cooperated more with other countries, the number of coronavirus cases would have been lower in this country
                (C) Many of the problems facing our country can be solved by working with other countries
                (D) Relying too much on military force to defeat terrorism creates hatred that leads to more terrorism
                (E) No amount of cooperation between (survey country) and other countries would have reduced the number of coronavirus cases in this country
                (F) Few of the problems facing our country can be solved by working with other countries
                (G) DK/Refused
                (H) When dealing with major international issues, our country should take into account the interests of other countries, even if it means making compromises with them
                (I) When dealing with major international issues, our country should follow its own interests, even when other countries strongly disagree
                output: 
                Multiple Choice

                Input: 
                question: When it comes to each of the following areas, do you think the United States is the best, above average, average, below average, or the worst? c. Standard of livingWhen it comes to each of the following areas, do you think the United States is the best, above average, average, below average, or the worst? c. Standard of living
                options: 
                (A) The best
                (B) Above average
                (C) Average
                (D) Below average
                (E) The worst
                (F) DK/Refused
                output:
                Likert scale

                Input: 
                question: Do you dress formally for work?
                options: None
                output:
                Binary Choice

                Input: 
                question: In which city was a famous playwright born?
                options: None
                output: 
                Free-text
                """
    prompt = f"\n\nquestion: {question}\n options: {options_formatted} \n output:"
    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content":  system
        },
        {
        "role": "user",
        "content":  prompt
        }
    ],
    max_tokens=10
    )
    #print(response.choices[0].message.content)
    # Extracting the similarity score from the response
    question_type = response.choices[0].message.content
    print(question_type)
    return question_type



In [140]:
data = pd.read_csv('data_merged_updated_question_types.csv', sep=';')


In [153]:
data.to_csv('data_merged_updated_question_types.csv', sep =';')

In [169]:
gpt_generated.columns

Index(['Unnamed: 0', 'question', 'selections', 'options', 'options_formatted',
       'source', 'value_us', 'value_uk', 'answer_us', 'answer_uk', 'category',
       '# of options', 'question type', 'category_group', 'model_answer_us',
       'model_answer_uk', 'model_answer_uk_option_match',
       'model_answer_us_option_match', '#_options', 'score_uk', 'score_us',
       'overall_score_uk', 'similarity_score_uk', 'similarity_score_us',
       'overall_score_us', 'similarity_model_answers_uk_us',
       'similarity_ground_truth_answers_uk_us', 'options_dict',
       'score_ground_truth_answers', 'value_diff', 'question_type',
       'gpt4_uk_score', 'gpt4_us_score', 'gpt4_usGT_ukGT_score',
       'gpt4_ukGT_usMA_score', 'gpt4_usGT_ukMA_score', 'gpt4_usMA_ukMA_score'],
      dtype='object')

In [170]:
def normalize_column(df, column_name, new_column_name):
    """
    Normalize the values in a DataFrame column to the range 0-1 and save them in a new column.
    
    Parameters:
    df (pandas.DataFrame): The DataFrame containing the column to normalize.
    column_name (str): The name of the column to normalize.
    new_column_name (str): The name of the new column for the normalized values.
    
    Returns:
    pandas.DataFrame: A DataFrame with the additional normalized column.
    """
    # Copy the DataFrame to avoid modifying the original data
    df_normalized = df.copy()
    
    # Apply Min-Max normalization
    df_normalized[new_column_name] = (df_normalized[column_name] - 1) / (5 - 1)
    
    return df_normalized

In [182]:
gpt_generated = normalize_column(gpt_generated, 'gpt4_us_score', 'gpt4_us_score_normed')
gpt_generated = normalize_column(gpt_generated, 'gpt4_uk_score', 'gpt4_uk_score_normed')
gpt_generated = normalize_column(gpt_generated, 'gpt4_ukGT_usMA_score', 'gpt4_ukGT_usMA_normed')
gpt_generated = normalize_column(gpt_generated, 'gpt4_usGT_ukMA_score', 'gpt4_usGT_ukMA_normed')
gpt_generated = normalize_column(gpt_generated, 'gpt4_usGT_ukGT_score', 'gpt4_usGT_ukGT_normed')
gpt_generated = normalize_column(gpt_generated, 'gpt4_usMA_ukMA_score', 'gpt4_usMA_ukMA_score_normed')


In [183]:
def add_small_value(df, column_name):
    """
    Add 0.00001 to each value in the specified column of a DataFrame 
    if the value is not 0.0 or 1.0.
    
    Parameters:
    df (pandas.DataFrame): The DataFrame containing the column.
    column_name (str): The name of the column to modify.
    
    Returns:
    pandas.DataFrame: A DataFrame with the modified column.
    """
    # Define the lambda function for the condition
    add_value = lambda x: x + 0.000001 if x not in [0.0, 1.0] else x

    # Apply the function to the specified column
    df[column_name] = df[column_name].apply(add_value)
    
    return df

In [184]:
gpt_generated = add_small_value(gpt_generated, 'gpt4_us_score_normed')
gpt_generated = add_small_value(gpt_generated, 'gpt4_uk_score_normed')
gpt_generated = add_small_value(gpt_generated, 'gpt4_ukGT_usMA_normed')
gpt_generated = add_small_value(gpt_generated, 'gpt4_usGT_ukMA_normed')
gpt_generated = add_small_value(gpt_generated, 'gpt4_usGT_ukGT_normed')
gpt_generated = add_small_value(gpt_generated, 'gpt4_usMA_ukMA_score_normed')



In [185]:
gpt_generated.to_csv('gpt_similarity_scores.csv', sep=';')

In [238]:
from efficiency.nlp import Chatbot
import os
import openai
import pandas as pd
openai.api_key = os.environ["OPENAI_API_KEY"]
chat = Chatbot(system_prompt="Act like if you were a real person and answer the question accordingly.", max_tokens=30)

quiz = data[(data['source'] == 'GPT / https://www.beelovedcity.com/england-quiz') | (data['source'] == 'GPT / https://www.anadventurousworld.com/usa-trivia-questions/')]

chars = pd.read_csv('../inspect/characters_firstPerson.csv', sep=';')

import openai
import os
import pandas as pd
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def quiz_answer(question, category_group, model="gpt-4"):
    """
    Measures the similarity between two texts using GPT-4.

    Parameters:
    text1 (str): First text for comparison.
    text2 (str): Second text for comparison.
    model (str): The GPT model to use.

    Returns:
    float: A similarity score between 0 (not similar) and 1 (very similar).
    """
    character = chars[chars['topic'] == category_group]['british_character'].iloc[0]
    query = character+ "\n"+" I have a question: " + question + "\nRemember, act as if you are a stereotypical person from my culture who knows the answer. Infer the culture from my way of speaking. Answer briefly with max 4 words. The less words the better."
    
    system = f"Act like if you were a real person and answer the question accordingly."
    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": system
        },
        {
        "role": "user",
        "content":  query
        }
    ],
    max_tokens=30
    )
    #print(response.choices[0].message.content)
    # Extracting the similarity score from the response
    model_answer_uk = response.choices[0].message.content
    print(model_answer_uk)
    return model_answer_uk


In [239]:
def apply_quiz_answer(row):
    try:
        return quiz_answer(row['question'], row['category_group'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

quiz['model_answer_uk'] = quiz.apply(apply_quiz_answer, axis=1)

The Lion.
Saint George
Red, white and blue.
Around 66 million people.
Six; Scotland, Wales, Ireland.
Winchester
Queen Elizabeth II
April 23rd, Saint George's
"God Save the Queen."
4 countries.
Ed Sheeran.
Liverpool, England.
Zanzibar, Tanzania.
David Bowie.
Geri Halliwell
Reginald Kenneth Dwight
Chris Martin.
Manchester, England
United Kingdom, England.
Winchester, England.
South Africa.
Charles Dickens
Stratford-upon-Avon.
Virginia Woolf
George Orwell
"Harry Potter series, love."
United Kingdom
Shrewsbury, England
Manchester, England.
London
Leavesden, Hertfordshire, England.
Greg Davies
Birmingham.
"The Archers"
"Dragon's Den"
Colin Firth.
Daniel Craig.
King George VI
Emilia Clarke.
Lake Windermere
The United States.
Stonehenge
Ben Nevis
The River Thames
Birmingham
Highland, Scotland
Manchester
Cornwall
Yorkshire Dales National Park
Tony Blair
University of Oxford.
Queen Elizabeth II
1666
The Blitz.
Westminster Abbey
Diana, 1997, Paris tunnel.
House of Lancaster
23rd June 2016
Queen 

/var/folders/tq/vxytrqv13kx3fp71v2wgtd000000gn/T/ipykernel_11855/1499489434.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quiz['model_answer_uk'] = quiz.apply(apply_quiz_answer, axis=1)


In [248]:
quiz = normalize_column(quiz, 'gpt4_us_score', 'gpt4_us_score_normed')
quiz = normalize_column(quiz, 'gpt4_uk_score', 'gpt4_uk_score_normed')
quiz = normalize_column(quiz, 'gpt4_ukGT_usMA_score', 'gpt4_ukGT_usMA_normed')
quiz = normalize_column(quiz, 'gpt4_usGT_ukMA_score', 'gpt4_usGT_ukMA_normed')
quiz = normalize_column(quiz, 'gpt4_usGT_ukGT_score', 'gpt4_usGT_ukGT_normed')
quiz = normalize_column(quiz, 'gpt4_usMA_ukMA_score', 'gpt4_usMA_ukMA_score_normed')


In [249]:
quiz = add_small_value(quiz, 'gpt4_us_score_normed')
quiz = add_small_value(quiz, 'gpt4_uk_score_normed')
quiz = add_small_value(quiz, 'gpt4_ukGT_usMA_normed')
quiz = add_small_value(quiz, 'gpt4_usGT_ukMA_normed')
quiz = add_small_value(quiz, 'gpt4_usGT_ukGT_normed')
quiz = add_small_value(quiz, 'gpt4_usMA_ukMA_score_normed')


In [250]:
quiz.to_csv('quiz_answers_new.csv', sep=';')

In [141]:
goqa_v2 = data[(data['source'] == 'GAS') | (data['source'] == 'WVS')]

In [165]:
# Score for questions with answer options
def calculate_score(row, language):
    if row['question type'] in ['Likert Scale', 'Numerical Scale', 'Ordinal Scale'] and row['#_options'] > 2:
        if language == "us": 
            ground_truth = row['answer_us']
            model_answer = row['model_answer_us_option_match']
        if language == "uk": 
            ground_truth = row['answer_uk']
            model_answer = row['model_answer_uk_option_match']
        options = row['options']

        # Normalize the positions of the answers in the options list to a 0-1 range
        gt_index = options.index(ground_truth) / (len(options) - 1)
        model_index = options.index(model_answer) / (len(options) - 1)

        # Calculate the absolute error
        error = abs(gt_index - model_index)

        # Score can be inversely related to the error (1 - error)
        score = 1 - error
        return score
    else: 
        if language == "us":
            return int(row['answer_us'] == row['model_answer_us_option_match'])
        elif language == "uk":
            return int(row['answer_uk'] == row['model_answer_uk_option_match'])


In [260]:
#goqa_v2['goqa_score_us'] = goqa_v2.apply(calculate_score, axis=1, language = "us")
#goqa_v2['goqa_score_uk'] = goqa_v2.apply(calculate_score, axis=1, language = "uk")

In [162]:
# Score for questions with answer options
def calculate_score_gt(row):
    if row['question type'] in ['Likert Scale', 'Numerical Scale', 'Ordinal Scale'] and row['#_options'] > 2:
 
        ground_truth = row['answer_uk']
        model_answer = row['answer_us']
        options = row['options']

        # Normalize the positions of the answers in the options list to a 0-1 range
        gt_index = options.index(ground_truth) / (len(options) - 1)
        model_index = options.index(model_answer) / (len(options) - 1)

        # Calculate the absolute error
        error = abs(gt_index - model_index)

        # Score can be inversely related to the error (1 - error)
        score = 1 - error
        return score
    else: 
        return int(row['answer_us'] == row['answer_uk'])


In [163]:
goqa_v2['goqa_ukGT_usGT_score'] = goqa_v2.apply(calculate_score_gt, axis=1)

/var/folders/tq/vxytrqv13kx3fp71v2wgtd000000gn/T/ipykernel_11855/3869438848.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  goqa_v2['goqa_ukGT_usGT_score'] = goqa_v2.apply(calculate_score_gt, axis=1)


In [193]:
goqa_v2 = add_small_value(goqa_v2, 'goqa_score_us')
goqa_v2 = add_small_value(goqa_v2, 'goqa_score_uk')
goqa_v2 = add_small_value(goqa_v2, 'goqa_ukGT_usMA_score')
goqa_v2 = add_small_value(goqa_v2, 'goqa_ukMA_usGT_score')
goqa_v2 = add_small_value(goqa_v2, 'goqa_ukGT_usGT_score')
goqa_v2 = add_small_value(goqa_v2, 'goqa_ukMA_usMA_score')


In [194]:
goqa_v2.to_csv('goqa_results.csv', sep=';')

In [267]:
data = pd.read_csv('data_merged_updated_question_types.csv', sep=';')


In [268]:
data.columns

Index(['Unnamed: 0', 'question', 'selections', 'options', 'options_formatted',
       'source', 'value_us', 'value_uk', 'answer_us', 'answer_uk', 'category',
       '# of options', 'question type', 'category_group', 'model_answer_us',
       'model_answer_uk', 'model_answer_uk_option_match',
       'model_answer_us_option_match', '#_options', 'options_dict',
       'score_ground_truth_answers', 'value_diff', 'question_type',
       'ukGT_usGT_score', 'ukMA_usGT_score', 'ukGT_usMA_score',
       'ukMA_usMA_score', 'us_score', 'uk_score'],
      dtype='object')

In [288]:
inspect = data[data['ukGT_usGT_score'] < 0.9]
print("-----------------")
print("US-Score:", inspect.us_score.mean())
print("UK-Score:", inspect.uk_score.mean())
print("GroundTruth-Score:", inspect.ukGT_usGT_score.mean())
print("UK-ModelAnswer_vs_US-GroundTruth-Score:", inspect.ukMA_usGT_score.mean())
print("US-ModelAnswer_vs_UK-GroundTruth-Score:", inspect.ukGT_usMA_score.mean())
print("UK-ModelAnswer_vs_US-ModelAnswer-Score:", inspect.ukMA_usMA_score.mean())
print("Data size:", len(inspect))
print("-----------------")

-----------------
US-Score: 0.6252447532807219
UK-Score: 0.5730409802750338
GroundTruth-Score: 0.4721716160356976
UK-ModelAnswer_vs_US-GroundTruth-Score: 0.5099368483347545
US-ModelAnswer_vs_UK-GroundTruth-Score: 0.5087425559716164
UK-ModelAnswer_vs_US-ModelAnswer-Score: 0.7475869767689469
Data size: 993
-----------------


In [270]:
inspect.source.value_counts()

source
GPT-4 generated                                                   557
GAS                                                               303
WVS                                                                84
GPT / https://www.beelovedcity.com/england-quiz                    33
GPT / https://www.anadventurousworld.com/usa-trivia-questions/     16
Name: count, dtype: int64

In [271]:
Us_similarity_scores_per_sample = inspect['us_score'].tolist()
Uk_similarity_scores_per_sample = inspect['uk_score'].tolist()
if_significantly_different(Us_similarity_scores_per_sample,Uk_similarity_scores_per_sample )

True

In [280]:
goqa = inspect[(inspect['source'] == 'GAS') | (inspect['source'] == 'WVS')]
print("-----------------")
print("US-Score:", goqa.us_score.mean())
print("UK-Score:", goqa.uk_score.mean())
print("GroundTruth-Score:", goqa.ukGT_usGT_score.mean())
print("Data size:", len(goqa))

-----------------
US-Score: 0.6705239147487259
UK-Score: 0.6599596909899446
GroundTruth-Score: 0.6307907822311312
Data size: 387


In [279]:
quiz = inspect[(inspect['source'] == 'GPT / https://www.beelovedcity.com/england-quiz') | (inspect['source'] == 'GPT / https://www.anadventurousworld.com/usa-trivia-questions/')]
print("US-Score:", quiz.us_score.mean())
print("UK-Score:", quiz.uk_score.mean())
print("GroundTruth-Score:", quiz.ukGT_usGT_score.mean())
print("Data size:", len(quiz))

US-Score: 0.4668369387755102
UK-Score: 0.6377552244897959
GroundTruth-Score: 0.16581659183673467
Data size: 49


In [278]:
gpt = inspect[(inspect['source'] == 'GPT-4 generated')]
print("US-Score:", gpt.us_score.mean())
print("UK-Score:", gpt.uk_score.mean())
print("GroundTruth-Score:", gpt.ukGT_usGT_score.mean())
print("Data size:", len(gpt))

US-Score: 0.6077204219030521
UK-Score: 0.5069574272890486
GroundTruth-Score: 0.388914486535009
Data size: 557


In [284]:
print("US-Score:", inspect.groupby(['category_group']).us_score.mean())
print("--------")
print("UK-Score:", inspect.groupby(['category_group']).uk_score.mean())
print("--------")
print("GroundTruth-Score:", inspect.groupby(['category_group']).ukGT_usGT_score.mean())


US-Score: category_group
Economy, Health, and Environment              0.595198
Lifestyle, Entertainment, and Daily Living    0.667428
Media, Technology, and Education              0.581688
Politics and Governance                       0.635872
Society, Culture, and Relationships           0.613374
Name: us_score, dtype: float64
--------
UK-Score: category_group
Economy, Health, and Environment              0.541285
Lifestyle, Entertainment, and Daily Living    0.525092
Media, Technology, and Education              0.561866
Politics and Governance                       0.664885
Society, Culture, and Relationships           0.580506
Name: uk_score, dtype: float64
--------
GroundTruth-Score: category_group
Economy, Health, and Environment              0.470626
Lifestyle, Entertainment, and Daily Living    0.392336
Media, Technology, and Education              0.369659
Politics and Governance                       0.631689
Society, Culture, and Relationships           0.490327
Name: ukGT_

In [285]:
print("US-Score:", inspect.groupby(['question_type']).us_score.mean())
print("--------")
print("UK-Score:", inspect.groupby(['question_type']).uk_score.mean())
print("--------")
print("GroundTruth-Score:", inspect.groupby(['question_type']).ukGT_usGT_score.mean())

US-Score: question_type
Binary Choice      0.590148
Free-text          0.592000
Likert Scale       0.722302
Multiple Choice    0.572743
Numerical Scale    0.404412
Ordinal Scale      0.792765
Name: us_score, dtype: float64
--------
UK-Score: question_type
Binary Choice      0.476207
Free-text          0.633000
Likert Scale       0.715677
Multiple Choice    0.541229
Numerical Scale    0.345589
Ordinal Scale      0.764691
Name: uk_score, dtype: float64
--------
GroundTruth-Score: question_type
Binary Choice      0.361083
Free-text          0.328001
Likert Scale       0.690425
Multiple Choice    0.453181
Numerical Scale    0.507354
Ordinal Scale      0.773860
Name: ukGT_usGT_score, dtype: float64


In [287]:
print("US-Score:", inspect.groupby(['source']).us_score.mean())
print("--------")
print("UK-Score:", inspect.groupby(['source']).uk_score.mean())
print("--------")
print("GroundTruth-Score:", inspect.groupby(['source']).ukGT_usGT_score.mean())

US-Score: source
GAS                                                               0.643831
GPT / https://www.anadventurousworld.com/usa-trivia-questions/    0.546875
GPT / https://www.beelovedcity.com/england-quiz                   0.428031
GPT-4 generated                                                   0.607720
WVS                                                               0.766808
Name: us_score, dtype: float64
--------
UK-Score: source
GAS                                                               0.630622
GPT / https://www.anadventurousworld.com/usa-trivia-questions/    0.375000
GPT / https://www.beelovedcity.com/england-quiz                   0.765152
GPT-4 generated                                                   0.506957
WVS                                                               0.765784
Name: uk_score, dtype: float64
--------
GroundTruth-Score: source
GAS                                                               0.601197
GPT / https://www.anadventurousworl